# Comparison Of Scenarios

In [1]:
import pandas as pd
import altair as alt
import numpy as np

## Import Scenario 1

In [2]:
def read_logs1(experiment:int):
    tx_logs = pd.read_csv(f"../logs/s1_v{experiment}_transmitter.csv", sep=';')
    rx_logs = pd.read_csv(f"../logs/s1_v{experiment}_receiver.csv", sep=';')
    
    tx_logs["ROLE"] = ["Tx"]*tx_logs.shape[0]
    rx_logs["ROLE"] = ["Rx"]*rx_logs.shape[0]
    
    tx_logs["EXPERIMENT"] = [experiment]*tx_logs.shape[0]
    rx_logs["EXPERIMENT"] = [experiment]*rx_logs.shape[0]
    
    tx_logs["SCENARIO"] = [1]*tx_logs.shape[0]
    rx_logs["SCENARIO"] = [1]*rx_logs.shape[0]
    
    return pd.concat([tx_logs, rx_logs], ignore_index=True)

In [3]:
def read_experiments1():
    experiments = pd.read_csv("../logs/s1_experiments.csv", sep=';')
    
    experiments["VELOCITY"] = (2*np.pi*experiments["RADIUS_M"]/experiments["DURATION_S"])*experiments["CYCLE"]
    
    experiments["SCENARIO"] = [1]*experiments.shape[0]
    
    return experiments

In [4]:
logs1 = pd.DataFrame([], columns=['SEQ', 'ACK', 'TIMESTAMP', 'PAYLOAD', 'RSSI', 'LQI', 'ROLE', 'EXPERIMENT'])

for experiment in range(1, 7):
    logs1 = pd.concat([logs1, read_logs1(experiment)], ignore_index=True)

tx_logs1 = logs1[logs1["ROLE"]=="Tx"]
rx_logs1 = logs1[logs1["ROLE"]=="Rx"]

experiments1 = read_experiments1()

logs_exps1 = pd.merge(logs1, experiments1, on=["EXPERIMENT", "SCENARIO"])

In [5]:
RTTs1 = pd.DataFrame((tx_logs1[tx_logs1["ACK"]==240].set_index(["SEQ", "EXPERIMENT"])["TIMESTAMP"] - tx_logs1[tx_logs1["ACK"]==0].set_index(["SEQ", "EXPERIMENT"])["TIMESTAMP"]))
RTTs1.columns=["RTT"]
RTTs1 = RTTs1.reset_index()
RTTs1["SCENARIO"] = [1]*RTTs1.shape[0]

In [6]:
RTTs1

,SEQ,EXPERIMENT,RTT,SCENARIO
0,4,1,7,1
1,5,1,6,1
2,6,1,9,1
3,7,1,4,1
4,8,1,10,1
...,...,...,...,...
243,51,6,8,1
244,52,6,7,1
245,53,6,9,1
246,54,6,5,1


## Import Scenario 2

In [7]:
def read_logs2(experiment:int):
    tx_logs = pd.read_csv(f"../logs/s2_v{experiment}_transmitter.csv", sep=';')
    tc_logs = pd.read_csv(f"../logs/s2_v{experiment}_transceiver.csv", sep=';')
    rx_logs = pd.read_csv(f"../logs/s2_v{experiment}_receiver.csv", sep=';')
    
    tx_logs["ROLE"] = ["Tx"]*tx_logs.shape[0]
    tc_logs["ROLE"] = ["TC"]*tc_logs.shape[0]
    rx_logs["ROLE"] = ["Rx"]*rx_logs.shape[0]
    
    tx_logs["EXPERIMENT"] = [experiment]*tx_logs.shape[0]
    tc_logs["EXPERIMENT"] = [experiment]*tc_logs.shape[0]
    rx_logs["EXPERIMENT"] = [experiment]*rx_logs.shape[0]
    
    tx_logs["SCENARIO"] = [2]*tx_logs.shape[0]
    tc_logs["SCENARIO"] = [2]*tc_logs.shape[0]
    rx_logs["SCENARIO"] = [2]*rx_logs.shape[0]
    
    return pd.concat([tx_logs, tc_logs, rx_logs], ignore_index=True)

In [8]:
def read_experiments2():
    experiments = pd.read_csv("../logs/s2_experiments.csv", sep=';')
    
    experiments["INNER_VELOCITY"] = (2*np.pi*experiments["INNER_RADIUS_M"]/experiments["DURATION_S"])*experiments["CYCLE"]
    experiments["OUTER_VELOCITY"] = (2*np.pi*experiments["OUTER_RADIUS_M"]/experiments["DURATION_S"])*experiments["CYCLE"] * (experiments["RELATIVE_MOVEMENT"] != "Tx stationary")
    experiments["STATIONARY_INNER_VELOCITY"] = experiments["INNER_VELOCITY"] * (experiments["RELATIVE_MOVEMENT"] == "Tx stationary")
    
    experiments["SCENARIO"] = [2]*experiments.shape[0]
    
    return experiments

In [9]:
logs2 = pd.DataFrame([], columns=['SEQ', 'ACK', 'TIMESTAMP', 'PAYLOAD', 'RSSI', 'LQI', 'ROLE', 'EXPERIMENT'])

for experiment in range(1, 6):
    logs2 = pd.concat([logs2, read_logs2(experiment)], ignore_index=True)

experiments2 =  read_experiments2()
logs_exps2 = pd.merge(logs2, experiments2, on=["EXPERIMENT", "SCENARIO"])

tx_logs2 = logs_exps2[logs_exps2["ROLE"] == "Tx"]
tc_logs2 = logs_exps2[logs_exps2["ROLE"] == "TC"]
rx_logs2 = logs_exps2[logs_exps2["ROLE"] == "Rx"]

In [10]:
logs_exps2

,SEQ,ACK,TIMESTAMP,PAYLOAD,RSSI,LQI,ROLE,EXPERIMENT,SCENARIO,PAYLOAD_LENGTH,INNER_RADIUS_M,OUTER_RADIUS_M,RELATIVE_MOVEMENT,DURATION_S,CYCLE,INNER_VELOCITY,OUTER_VELOCITY,STATIONARY_INNER_VELOCITY
0,5,0,2554,GMSCRNOIA,-84,107,Tx,1,2.0,10,15,30,in line,59.8,0.5,0.788025,1.57605,0.000000
1,5,240,2557,NaN,-79,108,Tx,1,2.0,10,15,30,in line,59.8,0.5,0.788025,1.57605,0.000000
2,5,255,2563,NaN,-80,107,Tx,1,2.0,10,15,30,in line,59.8,0.5,0.788025,1.57605,0.000000
3,6,0,2682,FTLFPCUQF,-80,107,Tx,1,2.0,10,15,30,in line,59.8,0.5,0.788025,1.57605,0.000000
4,6,240,2691,NaN,-89,107,Tx,1,2.0,10,15,30,in line,59.8,0.5,0.788025,1.57605,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2684,26,0,12282,VDPJZBKQEXISGXJTDNUNDTGADIVFKLKBLMVZDQY,-70,107,Rx,5,2.0,40,5,10,Tx stationary,23.6,1.0,1.331183,0.00000,1.331183
2685,27,255,12409,NaN,-75,107,Rx,5,2.0,40,5,10,Tx stationary,23.6,1.0,1.331183,0.00000,1.331183
2686,27,0,12409,VAWWJETYXZHQQELZOSDEKSXPJGAMABUCMHRQOOB,-75,107,Rx,5,2.0,40,5,10,Tx stationary,23.6,1.0,1.331183,0.00000,1.331183
2687,28,255,12538,NaN,-83,108,Rx,5,2.0,40,5,10,Tx stationary,23.6,1.0,1.331183,0.00000,1.331183


In [11]:
RTTs2 = pd.DataFrame((tx_logs2[tx_logs2["ACK"]==255].set_index(["EXPERIMENT", "SEQ"])["TIMESTAMP"] - tx_logs2[tx_logs2["ACK"]==0].set_index(["EXPERIMENT", "SEQ"])["TIMESTAMP"]))
RTTs2.columns=["RTT"]
RTTs2 = RTTs2.reset_index()

RTTs2["FROM"] = ["Tx"]*RTTs2.shape[0]
RTTs2["TO"] = ["Rx"]*RTTs2.shape[0]
RTTs2["SCENARIO"] = [2]*RTTs2.shape[0]

## Comparison

In [12]:
from typing import Tuple, List

def select_experiments(data:pd.DataFrame, sc_exps:List[Tuple]):
    return data[data[["SCENARIO", "EXPERIMENT"]].apply(tuple, 1).isin(sc_exps)]

experiments = pd.concat([experiments1.rename(columns={"RADIUS_M":"OUTER_RADIUS_M"}), experiments2])

In [13]:
experiments

,EXPERIMENT,MOVING_ROLE,PAYLOAD_LENGTH,OUTER_RADIUS_M,DURATION_S,CYCLE,VELOCITY,SCENARIO,INNER_RADIUS_M,RELATIVE_MOVEMENT,INNER_VELOCITY,OUTER_VELOCITY,STATIONARY_INNER_VELOCITY
0,1,Tx,10,10,39.0,1.0,1.611073,1,NaN,NaN,NaN,NaN,NaN
1,2,Rx,10,10,41.8,1.0,1.503154,1,NaN,NaN,NaN,NaN,NaN
2,3,Rx,40,10,37.5,1.0,1.675516,1,NaN,NaN,NaN,NaN,NaN
3,4,Rx,40,10,15.6,1.0,4.027683,1,NaN,NaN,NaN,NaN,NaN
4,5,Rx,40,30,52.5,0.5,1.795196,1,NaN,NaN,NaN,NaN,NaN
5,6,Rx,10,30,52.0,0.5,1.812457,1,NaN,NaN,NaN,NaN,NaN
0,1,NaN,10,30,59.8,0.5,NaN,2,15.0,in line,0.788025,1.576050,0.000000
1,2,NaN,40,30,68.8,0.5,NaN,2,15.0,in line,0.684940,1.369881,0.000000
2,3,NaN,40,30,74.7,1.0,NaN,2,20.0,Tx stationary,1.682245,0.000000,1.682245
3,4,NaN,40,30,43.5,1.0,NaN,2,10.0,Tx stationary,1.444410,0.000000,1.444410


In [14]:
RTTs = pd.concat([RTTs1, RTTs2[["EXPERIMENT", "SCENARIO", "RTT"]]])

In [15]:
RTTs

,SEQ,EXPERIMENT,RTT,SCENARIO
0,4.0,1,7,1
1,5.0,1,6,1
2,6.0,1,9,1
3,7.0,1,4,1
4,8.0,1,10,1
...,...,...,...,...
269,NaN,5,16,2
270,NaN,5,12,2
271,NaN,5,11,2
272,NaN,5,12,2


In [16]:
selected = select_experiments(RTTs, [(1, 3), (2, 5), (1, 5), (2, 2)])

alt.Chart(pd.merge(selected, experiments), title="Comparison of Round Trip Times In Scenarios").transform_calculate(
    sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
).mark_bar().encode(
    x=alt.X("sc_exp:N", title="Experiment"),
    y="average(RTT):Q",
    column="OUTER_RADIUS_M"
).configure_title(
    anchor='middle'
)

D:\Programme\Python\3.11\Lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [17]:
selected1 = select_experiments(RTTs, [(1, 3), (2, 5)])
selected2 = select_experiments(RTTs, [(1, 5), (2, 2)])
selected3 = select_experiments(RTTs, [(1, 6), (2, 1)])

alt.hconcat(
    
    alt.Chart(pd.merge(selected1, experiments)).transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).mark_bar().encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="average(RTT):Q"
    ),
    
    alt.Chart(pd.merge(selected2, experiments)).transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).mark_bar().encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="average(RTT):Q"
    ),
    
    alt.Chart(pd.merge(selected3, experiments)).transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).mark_bar().encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="average(RTT):Q"
    ),
    
    title="Comparison of Round Trip Times in Scenarios"
).configure_title(
    anchor='middle'
)

D:\Programme\Python\3.11\Lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [18]:
RSSIs1 = logs_exps1.loc[(logs_exps1["ACK"] == 0) & (logs_exps1["ROLE"] == "Rx")][["SCENARIO", "EXPERIMENT", "RSSI", "ROLE", "SEQ"]]

In [19]:
RSSIs2 = logs_exps2.loc[(logs_exps2["ACK"] == 0) & logs_exps2["ROLE"].isin(["Rx", "TC"])][["SCENARIO", "EXPERIMENT", "RSSI", "ROLE", "SEQ"]]
RSSIs2 = pd.merge(RSSIs2, RSSIs2, on=["SCENARIO", "EXPERIMENT", "SEQ"])
RSSIs2 = RSSIs2.loc[(RSSIs2["ROLE_x"] == "TC") & (RSSIs2["ROLE_y"] == "Rx")]
RSSIs2["RSSI"] = np.power(10, RSSIs2["RSSI_x"].div(10)) + np.power(10, RSSIs2["RSSI_y"].div(10))
RSSIs2["RSSI"] = 10*np.log10(np.array(RSSIs2["RSSI"], dtype=float))
RSSIs2

,SCENARIO,EXPERIMENT,RSSI_x,ROLE_x,SEQ,RSSI_y,ROLE_y,RSSI
2,2.0,1,-79,TC,5,-82,Rx,-77.235651
5,2.0,1,-79,TC,5,-82,Rx,-77.235651
11,2.0,1,-81,TC,6,-87,Rx,-80.026772
14,2.0,1,-81,TC,6,-87,Rx,-80.026772
20,2.0,1,-89,TC,7,-90,Rx,-86.460981
...,...,...,...,...,...,...,...,...
2397,2.0,5,-68,TC,26,-70,Rx,-65.875574
2403,2.0,5,-66,TC,27,-75,Rx,-65.485031
2406,2.0,5,-66,TC,27,-75,Rx,-65.485031
2412,2.0,5,-62,TC,28,-83,Rx,-61.965639


In [20]:
RSSIs = pd.concat([RSSIs1, RSSIs2])

In [21]:
selected1 = select_experiments(RSSIs, [(1, 3), (2, 5)])
selected2 = select_experiments(RSSIs, [(1, 5), (2, 2)])
selected3 = select_experiments(RSSIs, [(1, 6), (2, 1)])

alt.hconcat(
    
    alt.Chart(selected1).mark_bar().transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="average(RSSI)"
    ),
    
    alt.Chart(selected2).mark_bar().transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="average(RSSI)"
    ),
    
    alt.Chart(selected3).mark_bar().transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="average(RSSI)"
    ),
    
    title="Comparison of Received Signal Strength Indicator (RSSI) in Scenarios"
).configure_title(
    anchor='middle'
)

D:\Programme\Python\3.11\Lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [22]:
alt.hconcat(
    
    alt.Chart(selected1).mark_bar().transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="variance(RSSI)"
    ),
    
    alt.Chart(selected2).mark_bar().transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="variance(RSSI)"
    ),
    
    alt.Chart(selected3).mark_bar().transform_calculate(
        sc_exp = "'s' + datum.SCENARIO + ' e' + datum.EXPERIMENT"
    ).encode(
        x=alt.X("sc_exp:N", title="Experiment"),
        y="variance(RSSI)"
    ),
    
    title="Comparison of Received Signal Strength Indicator (RSSI) in Scenarios"
).configure_title(
    anchor='middle'
)

alt.HConcatChart(...)